In [3]:
# Importing Libraries
import numpy as np
from PIL import Image

# converting image to numpy array
image = Image.open("paper-plane.png") 
data = np.asarray(image) 
print(f'Convertion of Image to {type(data)}') 
print(f'The Dimension of image is : {data.shape}') 
h = data.shape[0]
w = data.shape[1]
c = data.shape[2]
print(f"Height of image : {h}")
print(f"Width of image : {w}")
print(f"Channel of image : {c}")

# Part I - Extract the energy out of the image.
def extract_energy(imgArray, to_show=False):
  h, w, c = imgArray.shape
  right = np.empty_like(imgArray) 
  right = np.concatenate((imgArray[1:,:,:], imgArray[h-1,:,:].reshape((1,-1,3))), axis=0)/255

  left = np.empty_like(imgArray) 
  left = np.concatenate((imgArray[0,:,:].reshape((1,-1,3)), imgArray[:h-1,:,:]), axis=0)/255
  
  down = np.empty_like(imgArray) 
  down = np.concatenate((imgArray[:,1:,:], imgArray[:,w-1,:].reshape((-1,1,3))), axis=1)/255
  
  up = np.empty_like(imgArray) 
  up = np.concatenate((imgArray[:,0,:].reshape((-1,1,3)), imgArray[:,:w-1,:]), axis=1)/255

  d_x = left - right
  d_y = up - down

  d_x = np.square(d_x)
  d_y = np.square(d_y)

  d_x = np.sum(d_x, axis=2) 
  d_y = np.sum(d_y, axis=2)

  d = d_x + d_y
  d = d / np.max(d, axis = 1).reshape(-1,1) *255
  if to_show:
    pix = d.astype(np.uint8)
    picture = Image.fromarray(pix) 
    picture.save('energy.png') 
  return d

# Get the matrix of the differentiation matrix
energy_matrix = extract_energy(data, True)

# Create a path starting from bottom to top using DP, which has less enengy in the energy martix by taking up left, up middle and up right in consideration.
def seam_path(data, energy_mat, to_show=False):
  h, w, _ = data.shape
  backTrack = np.zeros_like(energy_mat, dtype=np.int)
  energy_path = np.zeros_like(energy_mat, dtype=np.int)
  energy_path[h-1,:] = energy_mat[h-1,:]
  for i in reversed(range(h-1)):
    for j in range(w):
      if j == 0:
        energy_path[i,j] = energy_mat[i,j]+ min(energy_path[i+1,j], energy_path[i+1,j+1])
        backTrack[i,j] = j + np.argmin(energy_mat[i+1, j:j+2])
      elif j == w-1:
        energy_path[i,j] = energy_mat[i,j]+ min(energy_path[i+1,j], energy_path[i+1,j-1])
        backTrack[i,j] = j-1 +  np.argmin(energy_mat[i+1, j-1:j+1])
      else:
        energy_path[i,j] = energy_mat[i,j]+ min(energy_path[i+1,j-1], energy_path[i+1,j], energy_path[i+1,j+1])
        backTrack[i,j] = j-1 + np.argmin(energy_mat[i+1, j-1:j+2])
  if to_show:
    data_copy = data.copy()
    x = np.argmin(energy_path[0,:])
    data_copy[0,x,0] = 255
    data_copy[0,x,1] = 0
    data_copy[0,x,2] = 0
    for i in range(0,h-1):
      x = backTrack[i,x]
      data_copy[i+1,x,0] = 255
      data_copy[i+1,x,1] = 0
      data_copy[i+1,x,2] = 0
    pix = data_copy.astype(np.uint8)
    picture = Image.fromarray(pix) 
    picture.save('Seampath.png') 
  return energy_matrix, backTrack

# Get the energy matrix and path in backtrack matrix.
energy_mat, backTrack = seam_path(data, energy_matrix, True)

# remove that path from the image and resize the matrix.
def carve_image(data, energy_mat, backTrack):
  h, w, _ = data.shape
  mask = np.ones((h, w), dtype=np.bool)
  x = np.argmin(energy_mat[0,:])
  mask[0,x] = False
  for i in range(0,h-1):  
    x = backTrack[i,x]
    mask[i+1,x] = False 
  mask = np.stack([mask] * 3, axis=2)
  data = data[mask].reshape((h, w - 1, 3))
  return data

data = carve_image(data, energy_mat, backTrack)
print(data.shape)

# To carve columnwise.
def column_carve(data):
  h, w, _ = data.shape
  energy_image = extract_energy(data)
  energy_matrix, back_track = seam_path(data, energy_image)
  data = carve_image(data, energy_matrix, back_track)
  return data

def seam_carve(image, axis = 'r', dim = 50):
  data = np.asarray(image) 
  dimension = dim
  axis = axis

  # Height
  if axis == 'r':
    data = np.rot90(data, 1, (0, 1))
    for _ in range(dimension):
      data = column_carve(data)
    data = np.rot90(data, 3, (0, 1))

  # Width
  elif axis == 'c':
    for _ in range(dimension):
      data = column_carve(data)

  pix = data.astype(np.uint8)
  picture = Image.fromarray(pix) 
  picture.save('resized.png')

seam_carve(image, 'c', 10)

Convertion of Image to <class 'numpy.ndarray'>
The Dimension of image is : (128, 128, 4)
Height of image : 128
Width of image : 128
Channel of image : 4


ValueError: ignored

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))
  
  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

In [ ]:
from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))
  
  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))